In [1]:
import pandas as pd
import numpy as np
import ast
import random
import os
import sys
sys.path.append('../')
from modules import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)

In [4]:
# df = pd.read_csv('../../data/more_features/more_feats_correlated_noisy_6.csv')
# df = df.fillna(-1)
# class_dict = constants.CLASS_DICT
# df['label'] = df['label'].replace(class_dict)
# X = df.iloc[:, 0:-1]
# y = df.iloc[:, -1]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
# X_test_df = pd.concat([X_test, y_test], axis=1)
# X_test_df = X_test_df.reset_index(drop=True)
# X_test_df.head()
X_test_df = pd.read_csv('../data/18_jan/test_set_basic.csv')
X_test_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,1,0,1,0,0,...,1,0,0,1,0,1,0,0,0,1
1,1,0,0,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,1,0,1
2,1,0,1,1,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
3,1,1,0,0,0,1,0,0,1,0,...,0,1,1,1,0,0,0,0,0,1
4,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


#### The test df

In [19]:
test_df = pd.read_csv('../test_dfs/21_jan/dqn_test_df_13000000.csv')
test_df.head()

,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,score,is_success
0,0.0,17.0,1.0,1.0,1.0,"['cutaneous_lupus', 'delirium', 'joint_involve...",0.0,14.0,1.0
1,1.0,18.0,1.0,1.0,1.0,"['cutaneous_lupus', 'delirium', 'joint_involve...",False,17.0,True
2,2.0,19.0,1.0,1.0,1.0,"['cutaneous_lupus', 'delirium', 'joint_involve...",False,17.0,True
3,3.0,18.0,1.0,1.0,1.0,"['cutaneous_lupus', 'delirium', 'joint_involve...",False,13.0,True
4,4.0,18.0,1.0,0.0,0.0,"['cutaneous_lupus', 'delirium', 'low_c3', 'joi...",False,2.0,True


In [20]:
a1 = np.array(test_df['y_actual'], dtype=np.float32)
a2 = np.array(X_test_df['label'], dtype=np.float32)
assert np.array_equal(a1, a2)

In [21]:
combined_test_df = pd.concat([test_df, X_test_df], axis=1).reset_index(drop=True)
combined_test_df.head()

,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,score,is_success,ana,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0.0,17.0,1.0,1.0,1.0,"['cutaneous_lupus', 'delirium', 'joint_involve...",0.0,14.0,1.0,1,...,1,0,0,1,0,1,0,0,0,1
1,1.0,18.0,1.0,1.0,1.0,"['cutaneous_lupus', 'delirium', 'joint_involve...",False,17.0,True,1,...,1,0,0,0,0,0,0,1,0,1
2,2.0,19.0,1.0,1.0,1.0,"['cutaneous_lupus', 'delirium', 'joint_involve...",False,17.0,True,1,...,0,0,0,0,1,1,0,0,0,1
3,3.0,18.0,1.0,1.0,1.0,"['cutaneous_lupus', 'delirium', 'joint_involve...",False,13.0,True,1,...,0,1,1,1,0,0,0,0,0,1
4,4.0,18.0,1.0,0.0,0.0,"['cutaneous_lupus', 'delirium', 'low_c3', 'joi...",False,2.0,True,1,...,0,0,0,0,0,0,0,0,0,0


#### 0 - No Lupus

In [7]:
no_test_df = test_df[test_df.y_actual== 0]
no_x_test_df = X_test_df[X_test_df.label==0]
combined_no_test_df = pd.concat([no_test_df, no_x_test_df], axis=1)
combined_no_test_df.head()

,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,score,is_success,ana,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
4,4.0,18.0,1.0,0.0,0.0,"['cutaneous_lupus', 'delirium', 'low_c3', 'joi...",False,2.0,True,1,...,0,0,0,0,0,0,0,0,0,0
5,5.0,21.0,1.0,0.0,0.0,"['cutaneous_lupus', 'delirium', 'low_c3', 'joi...",False,6.0,True,1,...,0,0,0,0,0,0,0,1,0,0
6,6.0,18.0,1.0,0.0,0.0,"['cutaneous_lupus', 'delirium', 'joint_involve...",False,2.0,True,1,...,0,0,0,0,0,0,0,0,0,0
7,7.0,22.0,1.0,0.0,0.0,"['cutaneous_lupus', 'delirium', 'low_c3', 'joi...",False,8.0,True,1,...,1,0,0,0,0,0,0,0,0,0
8,8.0,24.0,1.0,0.0,0.0,"['cutaneous_lupus', 'delirium', 'low_c3', 'joi...",False,8.0,True,1,...,0,1,0,0,0,0,0,0,0,0


In [35]:
len(combined_no_test_df[combined_no_test_df.cutaneous_lupus==1]['trajectory'].unique())

29

In [15]:
combined_no_test_df[combined_no_test_df.cutaneous_lupus==1

0

In [12]:
combined_no_test_df.y_actual.value_counts()

0.0    6753
Name: y_actual, dtype: int64